<a href="https://colab.research.google.com/github/fscherers/ChefIA/blob/main/Colab_ChefIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip -q install google-genai
!pip install pytz
!pip install -q google-adk

In [22]:
import os
import pytz
import datetime
from google.colab import userdata

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

In [23]:
# --- Funções Auxiliares ---

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    """
    Envia uma mensagem a um agente e retorna sua resposta final.
    Cria uma nova sessão para cada chamada, não mantendo histórico entre chamadas
    diferentes.
    """
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *') # Ajuste para renderização Markdown mais consistente
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [24]:
# --- Definição dos Agentes ---

##########################################
# --- Agente 1: Investigador de Preferências do Usuário ---
##########################################
def agente_investigador(topico_inicial=None, data_e_hora_atual=None):
    investigador = Agent(
        name="agente_investigador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é um assistente culinário. Sua principal missão é coletar TODAS as informações necessárias do usuário para que outros agentes possam planejar uma receita e seu preparo.

        Você DEVE obter as seguintes 5 informações, de forma amigável, gentil, descolada, sucinta, uma informação de cada vez com perguntas diretas e pouquíssimas palavras:
        1.  **Ingredientes disponíveis:** Quais ingredientes a pessoa já possui em casa?
        2.  **Disposição para comprar:** A pessoa está disposta a comprar mais ingredientes? (Responda "Sim" ou "Não")
        3.  **Detalhes da compra (se sim):** Se a pessoa estiver disposta a comprar, quanto ela quer gastar com isso (orçamento)? E qual o **endereço completo** dela para as compras (incluindo **rua, número, bairro, cidade e estado**, se possível)? Você pode consultar as informações recebidas usando a busca do Google (google_search) para conferir se consegue localizar o endereço com precisão. É **CRUCIAL** que você obtenha um endereço o mais preciso possível. Se a informação inicial não for suficientemente precisa para uma busca de lojas ou cálculo de rota (ex: apenas bairro e cidade), **você DEVE solicitar ativamente mais detalhes, como rua e número**, antes de finalizar a conversa.
        4.  **Tempo de preparo:** Qual o tempo total que a pessoa tem disponível para preparar a comida (sem contar o tempo de compra, se aplicável)?
        5.  **Momento de preparo:** Quando a pessoa deseja fazer a receita?

        **Sua conduta:**
        * **Contexto da Conversa:** Você receberá o histórico da conversa até o momento. Use-o para entender o que já foi perguntado e respondido, e para identificar as informações que ainda faltam. Nunca se comunique de forma que a interpretação possa ser dúbia.
        * **Início da conversa:** Comece a conversa pedindo os ingredientes disponíveis ou uma ideia geral do que a pessoa quer cozinhar.
        * **Iteração:** A cada resposta do usuário, revise as 5 informações que você precisa.
            * **Se faltar alguma informação:** Pergunte CLARAMENTE e OBJETIVAMENTE o que ainda é necessário. Seja específico e não passe para a próxima etapa sem coletar o que falta. Não peça mais do que uma informação por vez. Seja SEMPRE gentil e divertido.
            Exemplo: "Ótimo! E quanto tempo você tem disponível para o preparo?".
            * **Se tiver todas as informações:** Responda com um resumo PRECISO de todas as 5 informações coletadas, iniciando com "FIM", seguindo o formato abaixo. Esta é a sua sinalização de que o trabalho de coleta está concluído.

        **Formato de Saída Final (APENAS QUANDO TIVER TUDO, NÃO RESPONDA NADA ALÉM DO QUE ESTÁ NO FORMATO DE SAÍDA):**
        FIM
        Ingredientes disponíveis: [Lista de ingredientes obtida, ex: "frango, arroz, cebola"]
        Disposto a comprar ingredientes: [Sim/Não]
        Orçamento para compras: [Valor em R$, ex: "R$ 50" ou "Não aplica"]
        Localização para compras: [Bairro - Cidade - ESTADO (ou endereço completo), ex: "Passo D'Areia - Porto Alegre - RS" ou "Av. Grécia, 887 - Passo D'Areia - Porto Alegre - RS" ou "Não aplica"]
        Tempo disponível para preparo: [Tempo total, ex: "1 hora e 30 minutos"]
        Data e hora desejada para preparo: [Data e hora, ex: "Hoje à noite, 19:00"]
        """,
        description="Agente que coleta e organiza informações do usuário para planejamento de receitas.",
        tools=[google_search]
    )
    return investigador

################################################
# --- Agente 2: Buscador de Receitas ---
################################################
def agente_buscador(informacoes_do_usuario):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é um agente culinário que vai receber informações do usuário e buscar receitas baseadas nelas.
        Considera que sal, óleo, açúcar e temperos como orégano e pimenta preta sempre estarão disponíveis para uso.
        Sua tarefa é aproveitar o máximo possível dos ingredientes disponíveis, tentando sempre usar todos eles.
        Pode ser bem criativo nos pratos, utilize a ferramenta de busca do Google (google_search) para conseguir mais pratos em sites de receitas.

        Você fará uma busca aprofundada usando a busca do Google (google_search) para encontrar receitas que contenham os ingredientes disponíveis.
        Selecione CINCO RECEITAS que melhor se enquadram nos requisitos recebidos. Observe atentamente:
        - O tempo disponível para preparo da receita.
        - A disponibilidade de compra de novos ingredientes e o orçamento da pessoa para isso.
        - Se algo não se enquadrar, a receita pode ser substituída por outra que se encaixe.

        Crie uma lista com as cinco receitas, contendo para cada uma delas:
        - Os ingredientes que a pessoa já possui e os que faltam (lista de compra necessária).
        - Locais comuns onde a compra pode ser feita (se houver), usando a ferramenta de busca do Google (google_search) para pesquisar nas proximidades do local informado.
        - Uma estimativa do tempo de preparo e sua dificuldade (fácil, médio, difícil), utilizando o Google (google_search) para pesquisar.
        - Uma breve descrição do prato, comparando os pontos positivos e negativos, levando em conta as informações recebidas e pesquisando com o Google (google_search) se necessário.
        """,
        description="Agente buscador de receitas",
        tools=[google_search]
    )
    entrada_para_o_buscador = informacoes_do_usuario
    receitas_encontradas = call_agent(buscador, entrada_para_o_buscador)
    return receitas_encontradas

######################################
# --- Agente 3: Criador de Roteiros de Compra ---
######################################
def agente_de_compra(informacoes_basicas_usuario, receitas_e_listas_de_compra):
    redator = Agent(
        name="agente_de_compra",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é um agente especializado em criar roteiros de compra inteligentes.
        Você receberá informações detalhadas do usuário (incluindo localização) e uma lista de receitas com suas respectivas listas de compra (ou a indicação de que nenhuma compra é necessária).

        Sua missão é:
        1.  **Analisar as receitas recebidas:** Para cada receita, verifique se uma lista de compra é necessária.
        2.  **Se NENHUMA compra for necessária:** Adicione uma observação simples ao final das informações daquela receita, indicando que "Nenhuma compra adicional é necessária para esta receita.".
        3.  **Se EXISTIR lista de compras:**
            * **Extrair Localização:** Use a 'Localização para compras' fornecida nas informações básicas do usuário como ponto de partida (origem) para o roteiro. Você pode deixar a localidade mais precisa usando a ferramenta de busca do Google (google_search).
            * **Pesquisar Lojas/Mercados:** Para os itens da lista de compras da receita, use a ferramenta de busca do Google (google_search) para tentar encontrar supermercados, mercearias ou mercados próximos à localidade do usuário que possam ter esses ingredientes. Priorize locais com boas avaliações ou menções de bons preços. Refine a busca.
            * **Estimar Trajeto e Tempo:** Para o(s) local(is) de compra identificado(s):
                * Use a ferramenta de busca do Google (google_search) para pesquisar e estimar o tempo de deslocamento DE IDA E VOLTA entre a localização do usuário e a(s) loja(s) de compra.
                * Faça essa estimativa para **três métodos de deslocamento**: a pé, de carro e de bicicleta. Ex: "Tempo estimado (ida e volta): Carro - 15 min, Bicicleta - 30 min, A pé - 1 hora."
            * **Estimar Valor Gasto:** Use a ferramenta de busca do Google (google_search) para pesquisar estimativas de preço para os itens da lista de compras em locais relevantes, ou para encontrar avaliações gerais sobre o custo-benefício de lojas específicas. Informe que a estimativa é aproximada. Ex: "Estimativa de custo dos itens na loja X: R$ [valor estimado]."
            * **Criar Roteiro Simples:** Apresente os locais de compra sugeridos e a estimativa de deslocamento/custo de forma clara e concisa para cada receita com lista de compras. Exiba uma estimativa de tempo total.
        4.  **Formato de Saída:** Retorne todas as informações das receitas originais, mas agora acrescidas dos detalhes do roteiro de compra (ou da indicação de não necessidade de compra) ao final de cada bloco de receita.

        **Lembre-se:** Você não tem acesso a um Google Maps API em tempo real, então suas estimativas de tempo e localização serão baseadas nos resultados que você encontrar com a ferramenta de busca do Google (google_search).
        """,
        description="Agente organizador de roteiros de compras baseado em lista de compras e localização do usuário",
        tools=[google_search]
    )
    entrada_para_o_agente_de_compra = (
        f"Informações básicas do usuário:\n{informacoes_basicas_usuario}\n\n"
        f"Receitas e listas de compra:\n{receitas_e_listas_de_compra}"
    )
    roteiros_de_compra = call_agent(redator, entrada_para_o_agente_de_compra)
    return roteiros_de_compra

##########################################
# --- Agente 4: Seletor de Receita ---
##########################################
def agente_seletor(informacoes_basicas_usuario, data_e_hora_atual, roteiros_de_compra_e_receitas):
    seletor_de_receita = Agent(
        name="agente_seletor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é um agente inteligente e muito atencioso, cuja tarefa principal é **selecionar a receita mais apropriada** para o usuário e gerar um **roteiro detalhado** de execução.

        Você receberá:
        1.  **Informações do usuário:** Detalhes sobre ingredientes disponíveis, disposição para comprar, orçamento, localização, tempo total disponível e momento desejado para o preparo.
        2.  **Data e hora atual:** O momento exato em que a decisão precisa ser tomada.
            **ATENÇÃO:** Esta data e hora é o ponto de partida **EXATO** para todos os seus cálculos e planejamento temporal. Use o dia da semana, dia, mês e ano fornecidos aqui fielmente.
        3.  **Receitas pré-selecionadas com roteiros de compra:** Uma lista de receitas, cada uma com seus ingredientes necessários, lista de compras (se houver), e os roteiros de compra (tempos de deslocamento e custos), ou a indicação de que não há compras necessárias.

        Sua conduta:
        * **Análise de Viabilidade:** Para CADA receita, avalie cuidadosamente sua viabilidade considerando:
            * **Ingredientes:** Quanto a receita aproveita dos ingredientes que o usuário já possui.
            * **Orçamento e Compras:** Se há itens a comprar e se o orçamento do usuário é suficiente.
            * **Tempo Total:** Se o tempo disponível do usuário é compatível com o tempo de preparo da receita *mais o tempo estimado de deslocamento para compras* (ida e volta, incluindo tempo na loja), caso haja compras.
            * **Disponibilidade Temporal (crucial!):** Compare a **data e hora atual** com o "Momento de preparo" desejado pelo usuário e os "horários de funcionamento" das lojas (se houver compras).
                Use a ferramenta debusca do Google (google_search) para **verificar os horários de abertura e fechamento** de lojas sugeridas (com base na localização do usuário e nos tipos de lojas que vendem os ingredientes).
                * Certifique-se de que há tempo hábil para ir e voltar das compras (se necessárias) *antes* que o preparo da receita comece ou antes que as lojas fechem.
                **REFORÇO:** Ao mencionar a data e hora em sua resposta, sempre use a data e o dia da semana conforme recebido na 'Data e hora atual'.

        * **Seleção da Melhor Receita:** Após a análise de viabilidade, selecione **apenas UMA** receita que melhor se encaixe em *todos* os critérios do usuário, priorizando a que:
            * Melhor se adapta ao tempo disponível e momento de preparo desejado.
            * Exija menos esforço ou custo em compras, se possível.
            * Seja realista para ser executada dadas as restrições de tempo e horário de funcionamento das lojas.
        * **Geração do Roteiro Final:** Para a receita selecionada, crie um **roteiro passo a passo** detalhado, desde o **momento atual** até o início do preparo da receita, incluindo:
            * **Horário de Início:** O momento atual (data e hora atual).
            * **Etapas de Compra (se necessárias):** Horário de saída para compras, método de transporte sugerido (o mais eficiente ou adequado ao tempo), locais sugeridos e horário estimado de retorno. (Lembre-se de utilizar a ferramenta de busca do Google (google_search) para validar as informações de horário de funcionamento dos estabeleciemntos).
            * **Etapas de Preparo:** Horário sugerido para iniciar o preparo da receita, tempo estimado de preparo.
            * **Horário Final:** O momento em que a receita estaria pronta.
            * Qualquer outra dica relevante para a execução do plano.

        **Formato de Saída:**
        Sua resposta final deve ser um texto claro e direto, apresentando:
        1.  A receita selecionada (Nome da Receita).
        2.  O roteiro detalhado do plano de execução (passo a passo com horários e ações, Ex.: "Às 18:00 de hoje vá a tal supermercado (Ida e volta estimada em X minutos a pé, Y minutos de carro e Z minutos de bicicleta)...").
        3.  O passo a passo da receita, como prepará-la.
        """,
        description="Agente que seleciona a melhor receita e cria um roteiro de execução detalhado, considerando tempo e disponibilidade.",
        tools=[google_search]
    )
    entrada_do_agente_seletor = (
        f"Informações do usuário:\n{informacoes_basicas_usuario}\n\n"
        f"Data e hora atual: {data_e_hora_atual}\n\n"
        f"Receitas disponíveis e roteiros de compra:\n{roteiros_de_compra_e_receitas}"
    )
    receita_selecionada_e_roteiro = call_agent(seletor_de_receita, entrada_do_agente_seletor)
    return receita_selecionada_e_roteiro



##########################################
# --- Agente 5: Revisor Culinário ---
##########################################
def agente_revisor_culinario(informacoes_para_revisao):
    revisor = Agent(
        name="agente_revisor_culinario",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é o Agente Revisor Culinário, sua tarefa final é pegar o roteiro completo da receita escolhida e o plano de execução, e apresentá-los ao usuário da forma mais clara, direta e envolvente possível. Lembre-se: as informações devem estar todas em Português do Brasil!

        Você receberá um texto contendo:
        - O nome da receita selecionada.
        - O roteiro detalhado de execução (incluindo compras, horários e deslocamentos).
        - O passo a passo de preparo da receita.

        Sua conduta:
        - Leia e compreenda totalmente as informações fornecidas.
        - Formate a saída de maneira que seja fácil de ler e seguir para o usuário final.
        - Use cabeçalhos claros e bullet points ou listas numeradas quando apropriado para organizar a informação.
        - Destaque as informações mais importantes (como o nome da receita, horários críticos e ingredientes principais).
        - Não adicione novas informações, nem faça novas buscas, nem altere o conteúdo factualmente. Seu objetivo é apenas otimizar a apresentação da informação já gerada.
        - Seja amigável, encorajador e útil em seu tom.

        **Formato de Saída:**
        Apresente a informação de forma estruturada e amigável, seguindo este esquema:

        Comece com uma introdução calorosa e entusiasmada, anunciando a receita escolhida.
        Em seguida, apresente o 'Seu Roteiro de Execução Detalhado' com os horários e passos de compra/preparo, incluindo a localidade dos estabelecimentos de compra.
        Por fim, apresente o 'Passo a Passo para o Preparo da Receita' com as instruções culinárias.
        Finalize com uma mensagem de encorajamento.

        Exemplo de Estrutura de Saída (adapte o conteúdo):
        ## Sua Receita Perfeita: [Nome da Receita Selecionada]!

        Que ótima escolha! Prepare-se para cozinhar algo delicioso com este plano super prático que preparamos para você:

        ---

        ### Seu Roteiro de Execução Detalhado

        * **[Horário de Início]**: [Ação]
        * **[Horário Estimado]**: [Ação]
        ...
        * **[Horário Estimado que a Refeição Estará Pronta]**: [Ação]

        ---

        ### Passo a Passo para o Preparo da Receita

        1.  [Primeiro passo]
        2.  [Segundo passo]
        ...
        [Último passo]

        ---

        Aproveite seu momento na cozinha! Boa sorte e bom apetite!
        """,
        description="Agente que revisa e formata as informações finais da receita e roteiro para apresentação clara ao usuário."
    )
    final_output = call_agent(revisor, informacoes_para_revisao)
    return final_output


In [32]:
# --- Fluxo Principal de Execução ---

# 1. Obtenção da Data e Hora Atual no Fuso Horário do Brasil
fuso_horario_brasil = pytz.timezone('America/Sao_Paulo')
# A boa prática é obter o horário UTC e depois converter para o fuso local desejado
data_e_hora_atual_tz = datetime.datetime.now(pytz.utc).astimezone(fuso_horario_brasil)
# Formate a data e hora para passar para os agentes, incluindo nome e offset do fuso
data_e_hora_formatada_para_agente = data_e_hora_atual_tz.strftime("%A, %d de %B de %Y, %H:%M:%S %Z%z")

# 2. Iniciar a Interação com o Agente Investigador
agente_de_coleta = agente_investigador() # Não precisa de parâmetros iniciais aqui
display(to_markdown(f"**E aí, mestre-cuca! 🧑‍🍳 Chega de sofrer pra decidir o rango, né?**\n\nEu sou o **ChefIA**, seu novo parceiro culinário inteligente, e tô aqui pra virar essa chave na sua cozinha! Bora transformar **o que você tem por aí** em pratos incríveis, sem estresse e com muito sabor?\n\nPra começar, qual a boa de hoje na sua despensa, geladeira, horta...?"))

informacoes_coletadas = "" # Variável para armazenar a resposta final do agente_investigador
historico_conversacao = []

while True:
    user_input = input("Você: ")
    historico_conversacao.append(f"Usuário: {user_input}")

    contexto_para_agente = "\n".join(historico_conversacao)
    agent_response = call_agent(agente_de_coleta, contexto_para_agente)
    historico_conversacao.append(f"Agente: {agent_response}")

    if agent_response.strip().startswith("FIM"):
        output_agente1 = agent_response.replace("FIM\n", "", 1).strip()
        print("\nInformações coletadas com sucesso!\n")
        display(to_markdown(output_agente1))
        break
    else:
        display(to_markdown(f"Agente: {agent_response}"))

display(to_markdown("\n**Trabalhando na sua solicitação. Aguarde, isso pode levar alguns instantes...**"))
# 3. Chamar o Agente Buscador de Receitas
#print("\n--- Próxima Etapa: Buscando Receitas ---")
receitas_sugeridas = agente_buscador(output_agente1)
#print(receitas_sugeridas)

# 4. Chamar o Agente Criador de Roteiros de Compra
#print("\n--- Próxima Etapa: Criando Roteiros de Compra ---")
roteiros_de_compra_finais = agente_de_compra(output_agente1, receitas_sugeridas)
#print(roteiros_de_compra_finais)

# 5. Chamar o Agente Seletor de Receita e Gerar Roteiro Final
#print("\n--- Próxima Etapa: Selecionando a Receita e Criando o Roteiro Final ---")

receita_e_roteiro_final = agente_seletor(
    informacoes_basicas_usuario=output_agente1,
    data_e_hora_atual=data_e_hora_formatada_para_agente,
    roteiros_de_compra_e_receitas=roteiros_de_compra_finais
)
# print(receita_e_roteiro_final) # Você pode comentar esta linha se quiser apenas a saída final do revisor

# 6. Chamar o Agente Revisor Culinário para Formatação Final
#print("\n--- Próxima Etapa: Preparando a Apresentação Final para Você! ---")
output_final_para_usuario = agente_revisor_culinario(receita_e_roteiro_final)
display(to_markdown(output_final_para_usuario))


> **E aí, mestre-cuca! 🧑‍🍳 Chega de sofrer pra decidir o rango, né?**
> 
> Eu sou o **ChefIA**, seu novo parceiro culinário inteligente, e tô aqui pra virar essa chave na sua cozinha! Bora transformar **o que você tem por aí** em pratos incríveis, sem estresse e com muito sabor?
> 
> Pra começar, qual a boa de hoje na sua despensa, geladeira, horta...?

Você: carré de porco, pimenta preta


> Agente: Beleza! Já anotei aqui o carré de porco e a pimenta preta. 😉
> 
> E aí, topa comprar mais algum ingrediente pra essa aventura culinária?


Você: sim


> Agente: Massa! E quanto você quer gastar no máximo pra completar os ingredientes? 🤔


...

> Agente: Beleza, 1 hora de preparo anotada! 😉
> 
> E pra fechar, quando você pretende colocar a mão na massa e preparar essa delícia? 🗓️⏰


Você: hoje 21h

Informações coletadas com sucesso!



> Ingredientes disponíveis: carré de porco, pimenta preta
> Disposto a comprar ingredientes: Sim
> Orçamento para compras: R$ 20
> Localização para compras: Rua Geremias Aires, 1313 - Setor Aeroporto - Porto Nacional - TO
> Tempo disponível para preparo: 1 hora
> Data e hora desejada para preparo: hoje 21h

> 
> **Trabalhando na sua solicitação. Aguarde, isso pode levar alguns instantes...**

> Olá! Que maravilha! Parece que encontramos a receita perfeita para o seu dia de hoje, encaixando direitinho no seu tempo e orçamento.
> 
> Como seu Agente Revisor Culinário, organizei todas as informações para que você tenha um roteiro claro e prático para preparar uma deliciosa **Bisteca (Carré) de Porco Rápida na Frigideira**!
> 
> ---
> 
> ## Sua Receita Perfeita: Bisteca (Carré) de Porco Rápida na Frigideira!
> 
> Que ótima escolha! Prepare-se para cozinhar algo delicioso com este plano super prático que preparamos para você. É rápido, saboroso e se encaixa perfeitamente no seu dia!
> 
> ---
> 
> ### Seu Roteiro de Execução Detalhado
> 
> *   **15:06 de hoje, Domingo, 18 de maio de 2025**: Momento atual.
> *   **15:15 de hoje, Domingo, 18 de maio de 2025**: Saia para fazer as compras necessárias (alho e manteiga - opcionais).
>     *   **Local Sugerido**: Um supermercado próximo no Setor Aeroporto (Rua Aires Joca, próximo ao Supermercado Rodrigues, se for conveniente) ou o Ideal Supermercado na região central de Porto Nacional. Lembre-se de verificar horários de domingo!
>     *   **Transporte Sugerido**: Bicicleta (20-30 min ida e volta) ou carro (10-15 min ida e volta) são ideais para otimizar o tempo. A pé (40-60 min ida e volta) exigirá agilidade na compra.
>     *   **Tempo na loja**: Estime 15-20 minutos para as compras.
>     *   **Horário estimado de retorno**: Dependendo do transporte, entre **15:40 (carro)** e **16:40 (a pé)**.
> *   **16:40 - 20:30 de hoje, Domingo, 18 de maio de 2025**: Tempo livre para relaxar e se preparar. Deixe os utensílios à mão para a hora de cozinhar!
> *   **20:30 de hoje, Domingo, 18 de maio de 2025**: Inicie o preparo da receita.
> *   **20:50 - 21:00 de hoje, Domingo, 18 de maio de 2025**: Sua Bisteca (Carré) de Porco Rápida na Frigideira estará pronta para ser servida!
> 
> ---
> 
> ### Passo a Passo para o Preparo da Receita
> 
> 1.  Tempere o carré de porco com sal, pimenta preta e orégano (se estiver usando). Certifique-se de cobrir todos os lados.
> 2.  Aqueça um fio de óleo em uma frigideira em fogo médio-alto. Se estiver usando manteiga (opcional), adicione-a junto com o óleo para dourar.
> 3.  Quando o óleo (e manteiga) estiver quente, coloque o carré na frigideira. Não coloque muitos pedaços ao mesmo tempo para não esfriar a frigideira. Doure bem os dois lados até obter uma crosta bonita.
> 4.  Reduza o fogo para médio e cozinhe por mais alguns minutos de cada lado, dependendo da espessura do carré e do ponto de cozimento desejado. Para saber se está pronto, você pode fazer um pequeno corte em um pedaço para verificar o interior.
> 5.  Se estiver usando alho, adicione-o picado nos últimos minutos para aromatizar, tomando cuidado para não queimar.
> 6.  Retire o carré da frigideira e deixe descansar por alguns minutos antes de servir. Isso ajuda a manter a suculência da carne.
> 7.  Sirva imediatamente.
> 
> ---
> 
> Aproveite seu momento na cozinha e o sabor delicioso que está por vir! Boa sorte e bom apetite!
